In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
import pickle
import os
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
import json
from collections import Counter
import logging

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.WARNING, datefmt='%I:%M:%S')
pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

import sys
sys.path.append('../')  # Add the parent directory to the system path
import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
import utils.autoencoder_utils as autils

%reload_ext autoreload
%autoreload 2

In [2]:
model = HookedTransformer.from_pretrained(
    "EleutherAI/pythia-70m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)

german_data = haystack_utils.load_json_data("data/german_europarl.json")
english_data = haystack_utils.load_json_data("data/english_europarl.json")

english_activations = {}
LAYER_TO_ABLATE = 3
NEURONS_TO_ABLATE = [669]
english_activations[LAYER_TO_ABLATE] = haystack_utils.get_mlp_activations(
    english_data[:100], LAYER_TO_ABLATE, model, mean=False
)
MEAN_ACTIVATION_INACTIVE = english_activations[LAYER_TO_ABLATE][
    :, NEURONS_TO_ABLATE
].mean()


def deactivate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_INACTIVE
    return value


deactivate_neurons_fwd_hooks = [
    (f"blocks.{LAYER_TO_ABLATE}.mlp.hook_post", deactivate_neurons_hook)
]

# Load bigrams
with open("./data/low_indirect_loss_trigrams.json", "r") as f:
    trigrams = json.load(f)

all_ignore, valid_tokens = haystack_utils.get_weird_tokens(model, plot_norms=False)
common_tokens = haystack_utils.get_common_tokens(
    german_data[:200], model, all_ignore, k=100
)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
data/german_europarl.json: Loaded 2000 examples with 152 to 2000 characters each.
data/english_europarl.json: Loaded 2000 examples with 165 to 2000 characters each.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [3]:
save_name = "49_fiery_mountain"  # "25_gallant_monkey"
model_name = "pythia-70m"
path = Path("pythia-70m")

with open(f"{model_name}/{save_name}.json", "r") as f:
    cfg = json.load(f)

cfg = autils.AutoEncoderConfig(
    cfg["layer"], cfg["act"], cfg["expansion_factor"], cfg["l1_coeff"]
)
cfg

AutoEncoderConfig(layer=4, act_name='hook_mlp_out', expansion_factor=8, l1_coeff=0.0008)

In [4]:
if cfg.act_name == "hook_mlp_out":
    d_in = model.cfg.d_model  # d_mlp
else:
    d_in = model.cfg.d_mlp
d_hidden = d_in * cfg.expansion_factor

encoder = AutoEncoder(d_hidden, cfg.l1_coeff, d_in)
encoder.load_state_dict(torch.load(os.path.join(path, save_name + ".pt")))
encoder.to(device)

AutoEncoder()

In [18]:
def eval_trigram_direction(trigram, random_prompts, encoder_neuron):
    correct_token_dla = autils.get_trigram_token_dla(
        model, encoder, encoder_neuron, trigram, cfg
    )
    (
        last_token_logit_encoded,
        last_token_logit_zeroed,
        last_token_logprob_encoded,
        last_token_logprob_zeroed,
        boosted_tokens,
        deboosted_tokens,
    ) = autils.get_direction_logit_and_logprob_boost(
        random_prompts,
        encoder,
        encoder_neuron,
        model,
        trigram,
        common_tokens,
        all_ignore,
        cfg,
    )
    # autils.print_direction_activations(german_data[:2], model, encoder, encoder_neuron, cfg)
    (
        context_active_loss,
        context_ablated_loss,
        feature_activation_context_active,
        feature_activation_context_inactive,
    ) = autils.get_context_effect_on_feature_activations(
        model,
        random_prompts,
        encoder,
        encoder_neuron,
        deactivate_neurons_fwd_hooks,
        cfg,
    )
    (
        encoder_context_active_loss,
        encoder_context_inactive_loss,
    ) = autils.get_encoder_token_reconstruction_losses(
        random_prompts, model, encoder, deactivate_neurons_fwd_hooks, cfg
    )
    (
        loss_encoder_direction_active,
        loss_encoder_direction_inactive,
        loss_encoder_direction_zeroed,
    ) = autils.get_encoder_feature_reconstruction_losses(
        random_prompts,
        encoder,
        model,
        encoder_neuron,
        feature_activation_context_active,
        feature_activation_context_inactive,
        cfg,
    )
    return {
        "Trigram": trigram,
        "Direction": encoder_neuron,
        "Correct token dla": correct_token_dla,
        "Last token logit encoded": last_token_logit_encoded,
        "Last token logit zeroed": last_token_logit_zeroed,
        "Last token logprob encoded": last_token_logprob_encoded,
        "Last token logprob zeroed": last_token_logprob_zeroed,
        "Context active loss": context_active_loss,
        "Context ablated loss": context_ablated_loss,
        "Encoder context active loss": encoder_context_active_loss.item(),
        "Encoder context inactive loss": encoder_context_inactive_loss.item(),
        "Encoder direction active loss": loss_encoder_direction_active.item(),
        "Encoder direction inactive loss": loss_encoder_direction_inactive.item(),
        "Direction activation context active": feature_activation_context_active,
        "Direction activation context inactive": feature_activation_context_inactive,
    }


def eval_trigram(trigram: str):
    # End in all three trigram tokens
    random_trigram_prompts = haystack_utils.generate_random_prompts(
        trigram, model, common_tokens, n=100, length=20
    )
    trigram_dla = autils.encoder_dla_batched(
        random_trigram_prompts, model, encoder, cfg
    )[:, -1].mean(0)
    encoder_neurons = autils.get_directions_from_dla(trigram_dla)
    # Don't contain last trigram token
    # dataset_trigram_prompts = autils.get_trigram_dataset_examples(model, trigram, german_data, max_prompts=100)
    data = []
    for encoder_neuron in encoder_neurons:
        res = eval_trigram_direction(trigram, random_trigram_prompts, encoder_neuron)
        data.append(res)
    return data


def eval_trigrams(trigrams: list[str]):
    data = []
    for trigram in tqdm(trigrams):
        data += eval_trigram(trigram)
    return data


data = eval_trigrams(trigrams)
df = pd.DataFrame(data)
df.to_csv(f"data/trigram_eval_{save_name}.csv")

  0%|          | 0/228 [00:00<?, ?it/s]

In [19]:
df.head()

,Trigram,Direction,Correct token dla,Last token logit encoded,Last token logit zeroed,Last token logprob encoded,Last token logprob zeroed,Context active loss,Context ablated loss,Encoder context active loss,Encoder context inactive loss,Encoder direction active loss,Encoder direction inactive loss,Direction activation context active,Direction activation context inactive
0,Abgeord,1762,0.555725,25.235693,23.479805,-2.461441,-1.736089,2.248441,4.811219,2.461417,2.670471,2.460343,2.373877,5.151664,4.392234
1,Abgeord,1749,0.815784,25.235693,24.718266,-2.461441,-2.601628,2.248441,4.811219,2.461417,2.670471,2.450845,2.506464,0.865864,0.527327
2,Abgeord,1580,0.341286,25.235693,24.702847,-2.461441,-2.285936,2.248441,4.811219,2.461417,2.670471,2.463660,2.486050,1.243480,1.404255
3,Anregung,1092,0.741654,24.177307,23.716904,-1.637715,-1.746295,1.815517,6.118330,1.637728,2.021789,1.636093,1.680937,1.026926,0.541619
4,Anregung,4072,0.524571,24.177307,23.356323,-1.637715,-1.854022,1.815517,6.118330,1.637728,2.021789,1.635981,1.753839,1.287795,0.499931


In [20]:
# Low loss increase from using encoder
df["Encoder loss increase"] = (
    df["Encoder direction active loss"] - df["Context active loss"]
).clip(lower=0)
# High loss increase when removing direction
df["Direction ablation loss increase"] = (
    df["Encoder direction inactive loss"] - df["Encoder direction active loss"]
).clip(lower=0)
# Look for low decoder loss increase but high direction ablation loss increase
df["Interest"] = df["Direction ablation loss increase"] - df["Encoder loss increase"]

In [21]:
boosted_logporb = df["Last token logprob encoded"] > df["Last token logprob zeroed"]
boosted_logit = df["Last token logit encoded"] > df["Last token logit zeroed"]
pos_dla = df["Correct token dla"] > 0
filtered_df = df[boosted_logporb & boosted_logit & pos_dla].copy()
print(len(df), len(filtered_df), sum(boosted_logporb), sum(boosted_logit), sum(pos_dla))

444 385 394 428 444


In [22]:
filtered_df.sort_values("Interest", ascending=False).head(20)

,Trigram,Direction,Correct token dla,Last token logit encoded,Last token logit zeroed,Last token logprob encoded,Last token logprob zeroed,Context active loss,Context ablated loss,Encoder context active loss,Encoder context inactive loss,Encoder direction active loss,Encoder direction inactive loss,Direction activation context active,Direction activation context inactive,Encoder loss increase,Direction ablation loss increase,Interest
186,innergeme,4072,1.157572,21.290022,16.642542,-3.099989,-6.182594,3.167385,5.402523,3.100044,4.449189,3.085035,3.690939,3.282212,2.547564,0.000000,0.605904,0.605904
266,Handelsver,584,1.118627,18.120243,15.997128,-2.905767,-4.615692,2.688336,5.865644,2.905791,3.953487,2.866349,3.577502,2.664534,1.429002,0.178014,0.711152,0.533139
10,China forder,62,1.345419,20.573227,14.025286,-1.399669,-6.900258,1.867226,4.890429,1.398350,2.367307,1.383643,1.885301,6.459126,5.351360,0.000000,0.501658,0.501658
374,isse forder,62,1.345419,21.364992,14.421988,-1.300429,-6.731775,1.781804,4.321985,1.300446,1.988589,1.292664,1.779058,6.286462,5.231203,0.000000,0.486395,0.486395
340,fer forder,62,1.345419,21.535933,14.893001,-1.214864,-6.132004,1.510983,3.983179,1.214860,2.006253,1.185222,1.666187,6.036119,4.929015,0.000000,0.480965,0.480965
188,innergeme,3197,0.832652,21.290022,20.563368,-3.099989,-3.590939,3.167385,5.402523,3.100044,4.449189,3.062783,3.541084,0.529961,0.047972,0.000000,0.478301,0.478301
382,ktion forder,62,1.345419,21.015951,14.548864,-1.446094,-6.514494,1.728582,4.491117,1.446125,1.951242,1.420306,1.888897,5.752903,4.808158,0.000000,0.468591,0.468591
273,agen forder,62,1.345419,21.917458,15.129575,-1.178817,-5.867035,1.520251,3.618384,1.178815,1.698394,1.166545,1.518345,6.298723,5.314038,0.000000,0.351800,0.351800
19,Franco forder,62,1.345419,22.303532,14.885261,-0.786667,-6.230092,1.400424,3.575026,0.786667,1.445746,0.764717,1.108495,6.745555,5.714249,0.000000,0.343777,0.343777
357,ige Abge,384,0.599724,22.893623,16.822409,-2.662203,-5.835008,2.695361,5.055069,2.662180,3.176529,2.664087,2.975869,5.693321,4.817931,0.000000,0.311782,0.311782
